In [14]:
import urllib.request as req, json, numpy as np 
import os
os.getcwd() 




'C:\\Users\\эльдо'

In [28]:
apikey = open("C:\\Users\\эльдо\\keys.txt").read()
req_template = "https://apidata.mos.ru/v1/{}/?api_key={}"

raw = req.urlopen(req_template.format("datasets/912/rows", apikey)).read()
with open("picnic.json", "wb") as f:
    f.write(raw)

with open("picnic.json", "rb") as f:
    raw = f.read()

picnic = [row["Cells"] for row in json.loads(raw)]

print("Всего мест", len(picnic))

Всего мест 156


In [30]:
latlon =  np.array([r["geoData"]["coordinates"] for r in picnic])
print("Координаты:", *latlon[:3], "...")
convenience = [
        (
            int(r["HasWifi"] != 'нет'), 
            int(r["HasToilet"] != 'нет'), 
            int(r["Paid"] == 'бесплатно')
        ) 
        for r in picnic]
print("Удобства:", "...", *convenience[145:150], "...")


Координаты: [37.5962793  55.90387218] [37.62439579 55.6041267 ] [37.46735372 55.65470589] ...
Удобства: ... (0, 0, 1) (0, 0, 1) (0, 0, 1) (0, 0, 1) (0, 0, 1) ...


In [32]:
def topN(expected, data, N):
    norms = list(map(lambda row, number: (np.linalg.norm(row - expected), number), 
            data, range(len(data)) ))      # пары (расстояние, индекс)
    norms.sort(key=lambda r: r[0])         # рейтинг
    # множество индексов
    return set(map(lambda r: r[1], norms[:N]))

In [36]:
me_geo = np.array([37.740783, 55.687649])  # широта, долгота
me_conv = np.array([1.0, 1.0, 1.0])        # хочу всё: wifi, туалет и чтобы бесплатно!

depth = 1
A, B = set(), set()
# пока в пересечении ничего нет - расширяем поиск
while not A & B: 
    A = topN(me_geo, latlon, depth)
    B = topN(me_conv, convenience, depth)
    depth += 1
    
print("Мы рассматриваем", depth, "ближайших и удобных мест. Самое интересное:")
for picnics in A & B:
    row = picnic[picnics]
    print(row["Address"])
    print("\tWifi: {}, WC: {}, Цена: {}".format(row["HasWifi"], row["HasToilet"], row["Paid"]))

Мы рассматриваем 19 ближайших и удобных мест. Самое интересное:
Липецкая улица, дом 10, корпус 2
	Wifi: да, WC: да, Цена: бесплатно


In [37]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 20})

closest = np.array([latlon[i] for i in A])
result = np.array([latlon[i] for i in A & B])
plt.figure(figsize=(10,7))
plt.plot(closest[:,0], closest[:,1], 'o', markersize=7, color='gray')
plt.plot(result[:,0], result[:,1], 'v', markersize=10, color='black')
plt.plot(*me_geo, '*', markersize=15, color='black')
plt.annotate("дом", xy=me_geo, xytext=(-15, -20), textcoords='offset points')
plt.annotate("каток!", xy=result[0], xytext=(-20, 10), textcoords='offset points')
plt.grid(True)
plt.xlabel('долгота')
plt.ylabel('широта')
plt.show()

<Figure size 1000x700 with 1 Axes>